In [42]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

today = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0).strftime("%Y%m%d")
today

'20250418'

## Import Connected TV TCs and add other device columns

In [43]:
# Initial Setup for merge later

df_connectedTV_raw = pd.read_csv("data/KOCOWA_4.0_tc_connectedTV.csv").reset_index(drop=True)
df_connectedTV_raw = df_connectedTV_raw.rename(columns={"FireTV": "Fire TV", "ROKU": "Roku", "AndroidTV": "Android TV", "AppleTV": "Apple TV"})
available_devices = ['Android Mobile', 'Apple Mobile', 'Android TV', 'Apple TV', 'Fire TV', 'Smart TV', 'Vizio TV', 'Roku', 'Web']

df_K4_connectedTV = df_connectedTV_raw.fillna(0).copy()
df_K4_connectedTV["Web"] = None
df_K4_connectedTV["Apple Mobile"] = None
df_K4_connectedTV["Android Mobile"] = None
df_K4_connectedTV["Smart TV"] = None
df_K4_connectedTV["Vizio TV"] = None
df_K4_connectedTV["project_name"] = "KOCOWA 4.0"

# Convert "UNTESTED" to True boolean flags
for index, row in df_K4_connectedTV.iterrows():
    for i in range(6, 10):  # Columns from index 6 to 9
        if row.iloc[i] == "UNTESTED":
            df_K4_connectedTV.iat[index, i] = True

df_K4_connectedTV = df_K4_connectedTV.rename(columns={
    "대분류": "main_category",
    "중분류": "sub_category",
    "소분류": "component",
    "Section": "scenario",
    "테스트 항목": "test_case",
    "기대결과": "expected_result"})

In [44]:
df_K4_connectedTV.columns

Index(['main_category', 'sub_category', 'component', 'scenario', 'test_case',
       'expected_result', 'Fire TV', 'Roku', 'Android TV', 'Apple TV', 'Web',
       'Apple Mobile', 'Android Mobile', 'Smart TV', 'Vizio TV',
       'project_name'],
      dtype='object')

In [45]:
df_K4_connectedTV
len(list(df_K4_connectedTV.main_category.unique()))

21

In [46]:
df_K4_connectedTV

,main_category,sub_category,component,scenario,test_case,expected_result,Fire TV,Roku,Android TV,Apple TV,Web,Apple Mobile,Android Mobile,Smart TV,Vizio TV,project_name
0,Splash Screen,0,0,Splash Screen,KOCOWA Logo image 확인,KOCOWA logo image 노출된다,True,True,True,True,None,None,None,None,None,KOCOWA 4.0
1,Splash Screen,0,0,Splash Screen,KOCOWA Logo image Animation 확인,KOCOWA logo image Animation 적용되어 노출된다,True,True,True,True,None,None,None,None,None,KOCOWA 4.0
2,Splash Screen,Get Started,0,Splash Screen>Get Started,Get Started Page 진입 동작 확인,Get Started Page 진입된다,True,True,True,True,None,None,None,None,None,KOCOWA 4.0
3,Splash Screen,Get Started,0,Splash Screen>Get Started,미결제 시 Initial Page 진입 동작 확인,Initial Page 진입된다,True,True,True,True,None,None,None,None,None,KOCOWA 4.0
4,Splash Screen,Get Started,0,Splash Screen>Get Started,Home Main Page 진입 동작 확인,Home Main Page 진입된다,True,True,True,True,None,None,None,None,None,KOCOWA 4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,General Page,Ratings Guidelines Screen,0,General Page>Ratings Guidelines Screen,QR Code 인식 시 동작 확인,Ratings Guidelines Webpage 진입된다,True,True,True,True,None,None,None,None,None,KOCOWA 4.0
1998,General Page,Help Center Screen,0,General Page>Help Center Screen,title text 확인,"""Help Center"" text 출력된다",True,True,True,True,None,None,None,None,None,KOCOWA 4.0
1999,General Page,Help Center Screen,0,General Page>Help Center Screen,sub text 확인,"""To check the contents of ‘Help Center’, pleas...",True,True,True,True,None,None,None,None,None,KOCOWA 4.0
2000,General Page,Help Center Screen,0,General Page>Help Center Screen,Help Center QR Code image 확인,QR Code image 노출된다,True,True,True,True,None,None,None,None,None,KOCOWA 4.0


## Group by main and sub categories to create `df_dev_component`

### test cases will be joined by ", "

In [47]:
device_columns = ['Fire TV', 'Roku', 'Android TV', 'Apple TV', 'Web','Apple Mobile', 'Android Mobile', 'Smart TV', 'Vizio TV']
col_orders = ['project_name','main_category', 'component', 'test_case', 'Fire TV', 'Roku', 'Android TV', 'Apple TV', 'Web','Apple Mobile', 'Android Mobile', 'Smart TV', 'Vizio TV']

# If component = 0, then fill it with sub_category
for index, row in df_K4_connectedTV.iterrows():
    if row.component == 0:
        if row.sub_category == 0:
            component = row.main_category
            df_K4_connectedTV.at[index, "sub_category"] = component
        else:
            component = row.sub_category
        df_K4_connectedTV.at[index, "component"] = component
        
# Grouping the dataframe by main_category, sub_category, and component
# Aggregating test_case by joining them with ", " and taking the maximum value for device_columns
df_dev_component = (
    df_K4_connectedTV
    .groupby(['project_name','main_category', 'component'])
    .agg({
        'test_case': lambda x: '\n'.join(x),  # Combine test cases into a single string
        **{col: 'max' for col in device_columns}  # Take the maximum value for each device column
    })
    .reset_index()  # Reset the index to make it a regular dataframe
).copy()
        
df_dev_component = df_dev_component[col_orders]

In [48]:
len(list(df_dev_component.main_category.unique()))

21

In [51]:
df_dev_component

,project_name,main_category,component,test_case,Fire TV,Roku,Android TV,Apple TV,Web,Apple Mobile,Android Mobile,Smart TV,Vizio TV
0,KOCOWA 4.0,Contents,Bookmarked Button,Bookmarked Button 확인\nBookmarked Button icon 확...,True,True,True,True,NaN,NaN,NaN,NaN,NaN
1,KOCOWA 4.0,Contents,Cast Section,title text 확인\nContent Cast 확인,True,True,True,True,NaN,NaN,NaN,NaN,NaN
2,KOCOWA 4.0,Contents,Continue Watching Button,Continue Watching Button 확인\nContinue Watching...,True,True,True,True,NaN,NaN,NaN,NaN,NaN
3,KOCOWA 4.0,Contents,Details Section,Content Released Year 확인\nGenre text 확인\nDivis...,True,True,True,True,NaN,NaN,NaN,NaN,NaN
4,KOCOWA 4.0,Contents,Director Section,title text 확인\nContent Cast 확인,True,True,True,True,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,KOCOWA 4.0,VARIETY Channel Page,Update Soon Screen,Speech Bubble icon 확인\nsub text 확인,True,True,True,True,NaN,NaN,NaN,NaN,NaN
230,KOCOWA 4.0,VARIETY Channel Page,VARIETY Channel Page,리모컨 아래 방향키 입력 시 동작 확인\n리모컨 위 방향키 입력 시 동작 확인\n리...,True,True,True,True,NaN,NaN,NaN,NaN,NaN
231,KOCOWA 4.0,WELCOME TO KOCOWA Page,Home Button,Home Button 확인\nHome Button stringset 확인\nHome...,True,True,True,True,NaN,NaN,NaN,NaN,NaN
232,KOCOWA 4.0,WELCOME TO KOCOWA Page,Subscribe Button,Subscribe Button 확인\nSubscribe Button stringse...,True,True,True,True,NaN,NaN,NaN,NaN,NaN


### Initial export for checkup (csv and excel): processed_KOCOWA_4.0_tc_connectedTV

In [50]:
df_dev_component.to_csv("data/processed/processed_KOCOWA_4.0_tc_connectedTV.csv", index=False)
# df_dev_component.to_excel("data/processed/processed_KOCOWA_4.0_tc_connectedTV.xlsx", index=False)